In [ ]:
!pip3 install transformers==4.39.1 peft==0.10.0

In [ ]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from peft import PeftModel

In [ ]:
# load the base model
base_model = "microsoft/phi-2"
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    attn_implementation="flash_attention_2",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=False)
tokenizer.add_tokens(["<|im_start|>", "<PAD>"])
tokenizer.pad_token = "<PAD>"
tokenizer.add_special_tokens(dict(eos_token="<|im_end|>"))
model.config.eos_token_id = tokenizer.eos_token_id

In [ ]:
# merge lora and base model
model = PeftModel.from_pretrained(model, "phi-2-cfr-lora")
model = model.merge_and_unload()

In [ ]:
# save
new_model = "phi-2-cfr"
model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)